In [1]:
from typing import List
import dask
import numpy as np
import dask.array as da
import dask.bag as bag
from dask.distributed import Client

In [2]:
# Note: if we were on a cluster we'd have to do more magic to install it on all the nodes in the cluster.
!pip install PyPDF2

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import fsspec

<Client: 'tcp://127.0.0.1:34711' processes=4 threads=8, memory=62.60 GiB>

In [4]:
client = Client()
client

/home/holden/.local/lib/python3.8/site-packages/distributed/node.py:183: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46275 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:46275/status,
Dashboard: http://127.0.0.1:46275/status,Workers: 4
Total threads: 8,Total memory: 62.60 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42905,Workers: 4
Dashboard: http://127.0.0.1:46275/status,Total threads: 8
Started: Just now,Total memory: 62.60 GiB
Comm: tcp://127.0.0.1:36039,Total threads: 2
Dashboard: http://127.0.0.1:35941/status,Memory: 15.65 GiB
Nanny: tcp://127.0.0.1:45879,


In [9]:
#tag::custom_load[]
def discover_files(path: str):
    (fs, fspath) = fsspec.core.url_to_fs(path)
    return (fs, fs.expand_path(fspath, recursive="true"))
def load_file(fs, file):
    """Load (and initially process) the data."""
    from PyPDF2 import PdfReader
    try:
        file_contents = fs.open(file)
        pdf = PdfReader(file_contents)
        return (file, pdf.pages[0].extract_text())
    except Exception as e:
        return (file, e)
def load_data(path: str):
    (fs, files) = discover_files(path)
    bag_filenames = bag.from_sequence(files)
    contents = bag_filenames.map(lambda f:load_file(fs, f))
    return contents
#end::custom_load[]

In [12]:
#tag::preprocess_json[]
def make_url(idx):
    page_size = 100
    start = idx * page_size
    return f"https://api.fda.gov/food/enforcement.json?limit={page_size}&skip={start}"
urls = list(map(make_url, range(0, 10)))
# Since they are multi-line json we can't use the default \n line delim
raw_json = bag.read_text(urls, linedelimiter="NODELIM")
def clean_records(raw_records):
    import json
    # We don't need the meta field just the results field
    return json.loads(raw_records)["results"]
cleaned_records = raw_json.map(clean_records).flatten()
# And now we can convert it to a DataFrame
df = bag.Bag.to_dataframe(cleaned_records)
#end::preprocess_json[]

In [13]:
df.head()

,country,city,address_1,reason_for_recall,address_2,product_quantity,code_info,center_classification_date,distribution_pattern,state,...,recalling_firm,recall_number,initial_firm_notification,product_type,event_id,more_code_info,recall_initiation_date,postal_code,voluntary_mandated,status
0,United States,Davie,4131 SW 47th Ave Ste 1403,Recall initiated as a precautionary measure du...,,"1,990 bottles","UPC No. 632687615989; Lot No. 30661601, Exp. D...",20161025,"FL, MI, MS, and OH.",FL,...,Pharmatech LLC,F-0276-2017,Letter,Food,75272,,20160808,33314-4036,Voluntary: Firm initiated,Ongoing
1,United States,Millbrae,375 Adrian Rd,"Mooncake products, manufactured and distribute...",,"2 cases (1 pc/bx, 48bx/cs)","FG-M1MOT-UW Best by Nov 1, 2016.",20170106,"CA, WA, OR.",CA,...,Magic Gourmet Trading Inc,F-0865-2017,"Two or more of the following: Email, Fax, Lett...",Food,75069,,20160831,94030-3104,Voluntary: Firm initiated,Terminated
2,United States,Miami,13439 NW 19 LANE,Virginia State (VDACS) found Listeria monocyto...,,144 pieces,UPC 635349 000390 Best By dates: 07/01/14 thr...,20141202,"FL, GA. NC, and TN",FL,...,"Oasis Brands, Inc",F-0609-2015,"Two or more of the following: Email, Fax, Lett...",Food,69516,NaN,20141010,33182,Voluntary: Firm initiated,Terminated
3,United States,Pompano Beach,2300 NW 19th St,FreshPoint South Florida is recalling sliced f...,,7 cases,Item # 302940.,20120808,Products were distributed in South Florida.,FL,...,FreshPoint South Florida,F-1922-2012,"Two or more of the following: Email, Fax, Lett...",Food,62750,,20120727,33069-5227,Voluntary: Firm initiated,Terminated
4,United States,Rancho Dominguez,2610 Homestead Pl,Firm was notified by supplier that Organic Gro...,,xx,Lot codes: 72746,20200413,"nationwide, Canada and Netherlands",CA,...,"Organic By Nature, Inc.",F-0904-2020,"Two or more of the following: Email, Fax, Lett...",Food,85253,,20200224,90220-5610,Voluntary: Firm initiated,Terminated


In [ ]:
files = []
files = discover_files("file:///tmp/a0")
list(files)

In [ ]:
load_data("file:///tmp/pdfs").compute()

In [ ]:
#tag::parallel_list[]
def parallel_recursive_list(path: str, fs = None) -> List[str]:
    print(f"Listing {path}")
    if fs is None:
        (fs, path) = fsspec.core.url_to_fs(path)
    info = []
    infos = fs.ls(path, detail=True)
    # Above could throw PermissionError, but if we can't list the dir it's probably wrong so let it bubble up
    files = []
    dirs = []
    for i in infos:
        if i["type"] == "directory":
            # You can speed this up by using futures, covered in "Advanced Scheduling"
            dir_list = dask.delayed(parallel_recursive_list)(i["name"], fs=fs)
            dirs += dir_list
        else:
            files.append(i["name"])
    for sub_files in dask.compute(dirs):
        files.extend(sub_files)
    return files
#end::parallel_list[]

In [ ]:
files = []
files = parallel_recursive_list("file:///tmp/a0")

In [ ]:
files

In [ ]:
#tag::parallel_list_large[]
def parallel_list_directories_recursive(path: str, fs = None) -> List[str]:
    """
    Recursively find all the sub directories.
    """
    if fs is None:
        (fs, path) = fsspec.core.url_to_fs(path)
    info = []
    # Ideally we could filter for directories here, but fsspec lacks that (for now)
    infos = fs.ls(path, detail=True)
    # Above could throw PermissionError, but if we can't list the dir it's probably wrong so let it bubble up
    dirs = []
    result = []
    for i in infos:
        if i["type"] == "directory":
            # You can speed this up by using futures, covered in "Advanced Scheduling"
            result.append(i["name"])
            dir_list = dask.delayed(parallel_list_directories_recursive)(i["name"], fs=fs)
            dirs += dir_list
    for sub_dirs in dask.compute(dirs):
        result.extend(sub_dirs)
    return result

def list_files(path: str, fs = None) -> List[str]:
    """List files at a given depth with no recursion."""
    if fs is None:
        (fs, path) = fsspec.core.url_to_fs(path)
    info = []
    # Ideally we could filter for directories here, but fsspec lacks that (for now)
    return map(lambda i: i["name"], filter(lambda i: i["type"] == "directory", fs.ls(path, detail=True)))
    

def parallel_list_large(path: str, npartitions = None, fs = None) -> bag:
    """
    Find all of the files (potentially too large to fit on the head node).
    """
    directories = parallel_list_directories_recursive(path, fs = fs)
    dir_bag = dask.bag.from_sequence(directories, npartitions = npartitions)
    return dir_bag.map(lambda dir: list_files(dir, fs = fs)).flatten()
#end::parallel_list_large[]

In [ ]:
parallel_list_large("/tmp/a0").compute()

In [ ]:
#tag::custom_load_nonfs[]
def special_load_function(x):
    ## Do your special loading logic in this function, like reading a database
    return ["Timbit", "Is", "Awesome"][0: x % 4]

partitions = bag.from_sequence(range(20), npartitions=5)
raw_data = partitions.map(special_load_function).flatten()
#end::custom_load_nonfs[]

In [ ]:
raw_data.take(5)

In [ ]:
bag.from_sequence(range(0,1000)).map(lambda x: (x, x)).foldby(lambda x, y: x + y, lambda x, y: x + y)